# Project 1 - Capital Asset Pricing Model (CAPM)

***Author: Luca Zanolo***

## Capital Asset Pricing Model (CAPM)

The CAPM formula is used to calculate the expected return on an asset or a portfolio based on its risk and its expected return in the market. The riskier the asset, the higher the expected return should be to compensate investors for taking on that risk. Higher-risk assets should have higher betas and higher expected returns, while lower-risk assets should have lower betas and lower expected returns.

***CAPM: E(r) = Alpha + Beta * (E(Rm) - fc)***

***Y = Alpha + Beta * X***

***Rp - Rf = Alpha + beta (Rm - Rf)***

***Rp - Rf = 0 -> Rp  = Rf***

- E(Rm): Expected return on the market. This is the return that investors expect to earn on a well-diversified portfolio that includes all assets in the market.
- E(r): Expected return on an asset or a portfolio. This is the return that investors expect to earn on their investment in the asset or portfolio.
- Rf: Risk-free rate of return. This is the rate of return that investors can earn on a risk-free asset, such as government bonds or treasury bills. It is considered risk-free because there is virtually no chance of default on these investments.

**Returns**: What an investor earns on a particular stock or market over a period of time.

**Risk-free rate**: The return an investor could earn with zero risk.

**Excess returns**: Return of a stock/market minus the risk-free return.

**Sharpe-ratio**: It's a ratio that compares the excess return of a stock (or portfolio) and the volatility(or standard deviation) of stock returns. It tells how much return i'm earning for each unit of risk that i'm taking. It can be calculated on the return or on the excess return (this choice should be same for numerator and denominator).
- *High*: investment is generating a higher return per unit of risk, and thus is more attractive to investors;
- *Low*: investment is generating a lower return per unit of risk, and may be less attractive to investors;
- *> 1*: it is a positive value for stock pefcormance;
- *> 2*: it is an excellent value for stock pefcormance;
- *< 0*: our stock is generating less returns than the risk-free rate.

**Stock STD / Volatility**: Indicates how much the stock's returns have varied from its average return over a given period of time.
- *High value of volatility*: stock's returns have been more volatile and unpredictable;
- *Low value of volatility*: stock's returns have been more stable and predictable.

**Variance of stock/market**: Misure of dispersion of the corresponding returns.
- *High*:  returns are more spread out and volatile;
- *Low*: returns are more consistent and less volatile.

**Covariance (stock/market)**: Misure the extent in which the stock and the market move together.
- *> 0*: stock's returns tend to move in the same direction as the market returns;
- *>> 0*: and stock's returns are highly influenced by the market
- *= 0*: No relationship between the stock's returns and the market returns
- *Low or < 0*: stock's returns tend to move in the opposite direction of the market, in other words, returns stock's returns are not strongly influenced by the market

**Correlation**: Strenght and direction of linear relationship between market and stock returns.
- *-1*: Stock moves in the opposite direction of the market
- *0*: No relationship between the stock and market returns.
- *+1*: Stock moves in the same direction as the market

**Alpha**: Measure of a security's pefcormance relative to its expected return based on its risk level. A portfolio manager who consistently generates positive alpha is considered to have superior investment skills, while a portfolio manager who consistently generates negative alpha is considered to have inferior investment skills. A positive alpha indicates that the security has outpefcormed the market or its benchmark, while a negative alpha indicates that the security has underpefcormed the market or its benchmark.
- *Alpha > 0*: stock or portfolio has generated a higher return than expected given its level of risk exposure;
- *Alpha < 0*: stock or portfolio has generated a lower return than expected given its level of risk exposure.
    
**Beta**: Measure of a stock's or a portfolio's systematic risk, which is the risk associated with the entire market. It measures how much the return of the asset or portfolio varies in response to changes in the market.
- *Beta = 1* : indicates that the asset's returns move in line with the market;
- *Beta > 1* : indicates that the asset is more volatile than the market;
- *Beta < 1* : indicates that the asset is less volatile than the market.


## Core Functions

### Used Modules

In [8]:
import pandas as pd
import numpy as np
import yfinance as yf
import getFamaFrenchFactors as gff
import statsmodels.api as sm
from tabulate import tabulate
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from fredapi import Fred
fred = Fred(api_key = '9d722abcde9cebf8cef3b4db21411253')

### Data Visualisation functions

In [9]:
# Functions for data visualization

def print_results(pf_stats, title ="\nEstimations for the portfolio: \n"):
    print(title)
    print(tabulate(pf_stats, headers = "keys",tablefmt='simple_grid', floatfmt=".4f"))

def print_datas(datas, title = "\nIntermediate datas: \n"):
    print(title)
    result = pd.concat([datas.head(5),datas.tail(5)], axis=0)
    print(tabulate(result.transpose(), headers = "keys", tablefmt='simple_grid', floatfmt=".4f"))
        
def chart(data1, data2, title1, title2, section_title):
    data1.index = [i.to_timestamp() for i in data1.index]
    data2.index = [i.to_timestamp() for i in data2.index]
    print("\n", section_title)
    
    # Data1 chart
    fig_prices = go.Figure()
    data1_mean = data1.mean(axis = 1)
    for col in data1.columns:
        fig_prices.add_trace(go.Scatter(x=data1.index, y=data1[col], name=col))
    fig_prices.add_trace(go.Scatter(x=data1_mean.index, y=data1_mean, name='Mean'))
    fig_prices.update_layout(title=title1)

    # Data2 chart
    data2_mean = data2.mean(axis = 1)
    fig_returns = go.Figure()
    for col in data2.columns:
        fig_returns.add_trace(go.Scatter(x=data2.index, y=data2[col], name=col))
    fig_returns.add_trace(go.Scatter(x=data2_mean.index, y=data2_mean, name='Mean'))
    fig_returns.update_layout(title=title2)
    fig_prices.show()
    fig_returns.show()
    
def chart_no_date(data, x_label, y_label, title):

    print("\n" + title)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data[x_label], y=data[y_label], mode='markers', name='Points'))
    fig.update_layout(title=title, xaxis_title=x_label, yaxis_title=y_label)
    fig.show()

### Data Extraction Functions

In [51]:
# Functions to download prices of stocks and fama/french factors

def get_datas(portfolio, mk_label, fcts_label, start_date_30, end_date):
    print("-> Starting downloading market and portfolio's components data ...")
    stocks = get_portfolio_stocks(portfolio, start_date_30, end_date)
    market = download_data(mk_label, start_date_30, end_date)
    factors = get_fama_french_3(fcts_label)
    print("-> Download Complete.\n")
    return market, stocks, factors

def download_data(data_label, start_date, end_date):
    data = pd.DataFrame(yf.download(tickers = data_label, start = start_date, end = end_date, interval='1mo', progress=False, prepost=True)['Close'])
    data.rename(columns={"Adj Close" : data_label}, inplace = True)
    data.index = pd.to_datetime(data.index)
    data.index = data.index.to_period('M')
    return data

# Download risk-free rate from Fama/French 3 in decimal format
def get_fama_french_3(*factors):
    factors_list = list(*factors)
    fc_data = pd.DataFrame(gff.famaFrench3Factor(frequency='m'))
    fc_data = fc_data[[col for col in fc_data.columns if col in factors_list or col == 'date_ff_factors']]
    fc_data.rename(columns={"date_ff_factors": 'Date'}, inplace=True)
    fc_data.set_index('Date', drop=True, inplace=True)
    fc_data.index = pd.to_datetime(fc_data.index)
    fc_data.index = fc_data.index.to_period('M')
    for factor in factors_list:
        fc_data[factor] = (fc_data[factor] * 100)
    return fc_data

def get_portfolio_stocks(pf, start_date_30, end_date):
    flag = True
    stocks_label = pf.keys()
    for st_label in stocks_label:
        stock = download_data(st_label, start_date_30, end_date)
        if flag:
            stocks = stock
            flag = False
        else:
            stocks = pd.merge(stocks, stock, left_index = True, right_index = True)
    return stocks

# Functions to calculate TERM and CREDIT factors.

def term_factor(start_date, end_date, term_label):
    # Download 10-year U.S. Treasury bond yield and 3-month U.S. Treasury bill yield data
    yield_10yr = fred.get_series('IRLTLT01USM156N')
    yield_3mo = fred.get_series('TB3MS')
    
    # Calculate TERM factor (10-year yield minus 3-month yield)
    term =  yield_10yr - yield_3mo 
    term.rename(term_label, inplace=True)
    output = pd.DataFrame(data = term).dropna()
    output.index = output.index.to_period('M')
    output = output.groupby(output.index).mean()
    return output

def credit_factor(start_date, end_date):
    # Download 10-year U.S. Treasury bond yield and 3-month U.S. Treasury bill yield data
    yield_baa = pd.DataFrame(web.DataReader('DBAA', 'fred', start_date, end_date))
    yield_aaa = pd.DataFrame(web.DataReader('DAAA', 'fred', start_date, end_date))
    yield_baa.index = pd.to_datetime(yield_baa.index)
    yield_aaa.index = pd.to_datetime(yield_aaa.index)
    yield_baa.index = yield_baa.index.to_period('M')    
    yield_aaa.index = yield_aaa.index.to_period('M')
    
    # Calculate TERM factor (10-year yield minus 3-month yield)
    yields = yield_baa.merge(yield_aaa, left_index=True, right_index=True, how='inner').dropna()
    yields['CREDIT'] =  yields['DBAA'] - yields['DAAA'] 
    output = pd.DataFrame(data = yields['CREDIT'])
    return output.groupby(output.index).mean()

### Statistics calculation functions

In [40]:
# Support functions

def calculate_ex_returns(pf, data, mk_label, rf_label):
    columns = [col for col in data if col != rf_label]
    for column in columns:
        if column != mk_label: # Return calculation for market is not weighted, since is not part of the portfolio
            data[column + '.Return'] = (((data[column] / data[column].shift(1)) - 1) * pf[column]) * 100  
            data[column + '.Exc.Return'] = data[column + '.Return'] - data[rf_label] * pf[column]
        else:
            data[mk_label + '.Return'] = ((data[mk_label] / data[mk_label].shift(1)) - 1) * 100 
            data[mk_label + '.Exc.Return'] = data[mk_label + '.Return'] - data[rf_label]
    with pd.option_context('mode.use_inf_as_na', True):
        data.dropna(inplace=True)
    return data

def normalize_data(data):
    norm_data = data.copy()
    for column in norm_data.columns:
        norm_data[column] = norm_data[column] / norm_data[column][0]
    return norm_data

def verify_weights(w):
        if sum([x*100 for x in w]) != 100:
            raise ValueError("Specified weight are not correctly distributed. In percentage, they must sum to 100 (Actual:", sum([x*100 for x in w]), "\n")
            

def get_portfolio_value(pf, balance, data):
    portfolio_value = 0
    st_labels = list(pf.keys())
    for stock in st_labels:
        stock_price = data[stock][-1]
        shares = (balance * pf[stock]) // stock_price
        portfolio_value += (shares * stock_price)
    return portfolio_value

In [41]:
# Used in Points: 1

def calculate_statistics_1f(pf, stocks, market, mk_label, RF, RF_label, index, start_date, end_date, balance):
    
    # Input operations
    verify_weights(list(pf.values()))   # Verifing weights and getting stocks label
    st_labels = list(pf.keys())
    col_st_ret = [col + '.Return' for col in st_labels]   # Creating label to access combined_datas(cb) DataFrame
    col_st_ex_ret = [col + '.Exc.Return' for col in st_labels]
    col_st_exp_ret = [col + '.Exp.Ret' for col in st_labels]
    mk_ret_label = mk_label + '.Return'
    mk_ex_ret_label = mk_label + '.Exc.Return'
    start_date = start_date.to_period('M')   # Adjusting input parameters
    end_date = end_date.to_period('M')
    stocks_n = normalize_data(stocks.loc[start_date : end_date, :])  
    market_n = normalize_data(market.loc[start_date : end_date, :])
    #RF_n = normalize_data(RF.loc[start_date : end_date, :])
    #------------------------------------------------------------
    
    cb = stocks_n.merge(RF, left_index=True, right_index=True, how='inner')   # Join all dataset for calculation (Cardinality and forced dateperiod match)
    cb = cb.merge(market_n, left_index=True, right_index=True, how='inner') 
    
    cb = calculate_ex_returns(pf, cb, mk_label, RF_label)   # Calculating stock/s and market returns and excess returns

    st_ret_avg = cb.filter(items = col_st_ret, axis = 1).sum(axis = 1).mean()   # Calculating average return for stock/s and market
    mk_ret_avg = cb[mk_ret_label].mean()

    st_ex_ret_avg = cb.filter(items = col_st_ex_ret, axis = 1).sum(axis = 1).mean()   # Calculating Average excess returns of stock/s, market and factor
    mk_ex_ret_avg = cb[mk_ex_ret_label].mean()
    RF_avg = cb[RF_label].mean()   # Average of factor

    # 2 - Calculating sharpe-ratio        
    st_ex_ret_std = cb.filter(items = col_st_ex_ret, axis = 1).std().mean()   # Standard dev. of stock excess return (Volatility)
    mk_ex_ret_std = cb[mk_ex_ret_label].std().mean()   # Standard dev. of market excess return (Volatility)
    st_sharpe_ratio = st_ex_ret_avg / st_ex_ret_std   # Stock sharpe-ratio
    
    # Stock/s and market covariance, variance and correlation

    st_mk_ex_ret_cov = cb.filter(items = col_st_ex_ret, axis = 1).iloc[:-1,-1].cov(cb[mk_ex_ret_label])   # Covariance of stock returns & market returns
    st_ex_ret_var =  cb.filter(items = col_st_ex_ret, axis = 1).var().mean()   # Variance of stock returns
    mk_ex_ret_var = cb[mk_ex_ret_label].var().mean()   # Variance of market returns
    st_mk_ex_ret_corr = st_mk_ex_ret_cov / (st_ex_ret_std * mk_ex_ret_std)   # Correlation of stock and market
        
    # 3,4 - Calculating Alpha and Beta 
    pf_exp_ret, beta, alpha = 0.0, 0.0, 0.0
    for col in st_labels:
        y = cb[str(col + '.Exc.Return')].values # Indipendent variable of OLS
        x = sm.add_constant(cb[mk_ex_ret_label].values) # Didipendent variable of OLS
        model = sm.OLS(y, x)
        results = model.fit()
        beta += results.params[1]
        alpha += results.params[0]
    #exp_ret = cb.filter(items = col_st_ret).sum().mean()
    
    # Output operations------------------------------------------
    output_stats = {
        'Start': cb.first_valid_index(),
        'End': cb.last_valid_index(),
        'Pf.Avg.Ret.%': st_ret_avg, 'Mk.Avg.Ret.%': mk_ret_avg,
        'Pf.Avg.Exc.Ret.%': st_ex_ret_avg, 'Mk.Avg.Exc.Ret.%': mk_ex_ret_avg,
        'RF.Avg': RF_avg,
        'Pf.Sharpe-ratio': st_sharpe_ratio,
        'Alpha.%': alpha, 'Beta': beta,
        'Pf.Std.%': st_ex_ret_std, 'Mk.Std.%': mk_ex_ret_std,
        'Pf.Var': st_ex_ret_var, 'Mk.Var': mk_ex_ret_var,
        'Cov': st_mk_ex_ret_cov,
        'Corr': st_mk_ex_ret_corr,
        #'Expected.Ret.%': exp_ret
    }
    
    #output_stats['St.PF.Value'] = balance
    #output_stats['Est.Act.PF.value'] = balance + (balance * ((output_stats['Expected.Ret.%'])))
    #output_stats['Est.Act.Profit'] = output_stats['Est.Act.PF.value'] - balance
    cb = cb.merge(stocks, left_index=True, right_index=True, how='inner', suffixes=('_Normalized', '_Real')) #Combining data togheter
    cb = cb.merge(market, left_index=True, right_index=True, how='inner', suffixes=('_Normalized', '_Real'))
    df = pd.DataFrame(data = output_stats, index = [index])
    return df, cb.reindex(columns=sorted(cb.columns, reverse=True))


In [42]:
# Used in Points:  2, 3, 4, 5(To compare results)

def calculate_statistics_2f(pf, stocks, market, mk_label, RF, RF_label, index, start_date, end_date, balance):
    
    # Input operations
    verify_weights(list(pf.values()))   # Verifing weights and getting stocks label
    st_labels = list(pf.keys())
    col_st_ret = [col + '.Return' for col in st_labels]   # Creating label to access combined_datas(cb) DataFrame
    col_st_ex_ret = [col + '.Exc.Return' for col in st_labels]
    mk_ret_label = mk_label + '.Return'
    mk_ex_ret_label = mk_label + '.Exc.Return'
    start_date = start_date.to_period('M')   # Adjusting input parameters
    end_date = end_date.to_period('M')
    stocks_n = normalize_data(stocks.loc[start_date : end_date, :])  
    market_n = normalize_data(market.loc[start_date : end_date, :])
    RF_n = normalize_data(RF.loc[start_date : end_date, :])
    #------------------------------------------------------------
    
    cb = stocks_n.merge(RF, left_index=True, right_index=True, how='inner')   # Join all dataset for calculation (Cardinality and forced dateperiod match)
    cb = cb.merge(market_n, left_index=True, right_index=True, how='inner') 
    
    cb = calculate_ex_returns(pf, cb, mk_label, RF_label)   # Calculating stock/s and market returns and excess returns

    st_ret_avg = cb.filter(items = col_st_ret, axis = 1).sum(axis=1).mean()   # Calculating average return for stock/s and market
    mk_ret_avg = cb[mk_ret_label].mean()

    st_ex_ret_avg = cb.filter(items = col_st_ex_ret, axis = 1).sum(axis=1).mean()   # Calculating Average excess returns of stock/s, market and factor
    mk_ex_ret_avg = cb[mk_ex_ret_label].mean()
    RF_avg = cb[RF_label].mean()   # Average of factor

    # Calculating sharpe-ratio        
    st_ex_ret_std = cb.filter(items = col_st_ex_ret, axis = 1).std().sum()   # Standard dev. of stock excess return (Volatility)
    mk_ex_ret_std = cb[mk_ex_ret_label].std().sum()   # Standard dev. of market excess return (Volatility)
    st_sharpe_ratio = st_ex_ret_avg / st_ex_ret_std   # Stock sharpe-ratio
    
    # Stock/s and market covariance, variance and correlation
    selected_cols = col_st_ex_ret + [mk_ex_ret_label]
    st_mk_ex_ret_cov = cb.loc[:, selected_cols].cov().iloc[:-1, -1].sum()
    st_ex_ret_var =  cb.filter(items = col_st_ex_ret, axis = 1).sum().var()   # Variance of stock returns
    mk_ex_ret_var = cb[mk_ex_ret_label].var()  # Variance of market returns
    st_mk_ex_ret_corr = st_mk_ex_ret_cov / (st_ex_ret_std * mk_ex_ret_std)   # Correlation of stock and market
        
    # Calculating Alpha and Beta 
    pf_exp_ret, beta, alpha = 0.0, 0.0, 0.0
    for col in st_labels:
        y = cb[str(col + '.Exc.Return')].values # Indipendent variable of OLS
        x = sm.add_constant(cb[mk_ex_ret_label].values) # Didipendent variable of OLS
        model = sm.OLS(y, x)
        results = model.fit()
        beta += results.params[1]
        alpha += results.params[0]
    
    #cb['Exp.Ret'] = cb[RF_label] + beta * (mk_ex_ret_avg - cb[RF_label])    # Portfolio expected return
    #exp_ret = cb['Exp.Ret'].sum()  
    
    # Output operations------------------------------------------
    output_stats = {
        'Start': cb.first_valid_index(),
        'End': cb.last_valid_index(),
        'Pf.Avg.Ret.%': st_ret_avg, 'Mk.Avg.Ret.%': mk_ret_avg,
        'Pf.Avg.Exc.Ret.%': st_ex_ret_avg, 'Mk.Avg.Exc.Ret.%': mk_ex_ret_avg,
        'RF.Avg': RF_avg,
        'Pf.Sharpe-ratio': st_sharpe_ratio,
        'Alpha.%': alpha, 'Beta': beta,
        'Pf.Std.%': st_ex_ret_std, 'Mk.Std.%': mk_ex_ret_std,
        'Pf.Var': st_ex_ret_var, 'Mk.Var': mk_ex_ret_var,
        'Cov': st_mk_ex_ret_cov,
        'Corr': st_mk_ex_ret_corr,
        #'Expected.Ret.%': exp_ret
    }
    
    #output_stats['St.PF.Value'] = balance
    #output_stats['Est.Act.PF.value'] = balance + (balance * (output_stats['Expected.Ret.%']))
    #output_stats['Est.Act.Profit'] = output_stats['Est.Act.PF.value'] - balance
    
    cb = cb.merge(stocks, left_index=True, right_index=True, how='inner', suffixes=('_Normalized', '_Real')) #Combining data togheter
    cb = cb.merge(market, left_index=True, right_index=True, how='inner', suffixes=('_Normalized', '_Real'))
    df = pd.DataFrame(data = output_stats, index = [index])
    return df, cb.reindex(columns=sorted(cb.columns, reverse=True))


In [43]:
# Mkt-RF factor from Fama/French 3 - Used in Point 5

def calculate_statistics_3f(pf, stocks, mkt_rf, mkt_rf_label, RF, RF_label, index, start_date, end_date, balance):
    
    # Verifing weights and getting stocks label
    verify_weights(list(pf.values()))
    st_labels = list(pf.keys())
    
    # Adjusting input
    start_date = start_date.to_period('M')
    end_date = end_date.to_period('M')
    stocks_n = normalize_data(stocks.loc[start_date : end_date, :])
    
    # Creating labels to access cb DataFrame
    col_st_ret = [col + '.Return' for col in st_labels]
    col_st_ex_ret = [col + '.Exc.Return' for col in st_labels]
    
    cb = stocks_n.merge(mkt_rf, left_index=True, right_index=True, how='inner') #Combining data togheter
    cb = cb.merge(RF, left_index=True, right_index=True, how='inner') #Combining data togheter
    
    # 0 - Calculating stocks returns and excess returns
    for column in st_labels:
            cb[column + '.Return'] = ((cb[column] / cb[column].shift(1)) - 1 ) * 100 * pf[column]
            cb[column + '.Exc.Return'] = cb[column + '.Return'] - (cb[RF_label]) * pf[column]
    with pd.option_context('mode.use_inf_as_na', True):   # Dropping columns with NaN or inf values
        cb.dropna(inplace=True)
    
    # Calculating average return for stock/s and market
    st_ret_avg = cb.filter(items = col_st_ret, axis = 1).sum(axis = 1).mean()
    mkt_rf_avg = cb[mkt_rf_label].mean()
    RF_avg = cb[RF_label].mean()   # Average of factor

    # Calculating Average excess returns of stock and market
    st_ex_ret_avg = cb.filter(items = col_st_ex_ret, axis = 1).sum(axis = 1).mean() # Average stock excess return
    
    # Calculating sharpe-ratio
    st_ex_ret_std = cb.filter(items = col_st_ex_ret, axis = 1).std().sum() # Standard dev. of stock excess return (Volatility)
    mkt_rf_std = cb[mkt_rf_label].std().sum() # Standard dev. of market excess return (Volatility)
    st_sharpe_ratio = st_ex_ret_avg / st_ex_ret_std # Stock sharpe-ratio
    
    # Other statistics
    selected_cols = col_st_ex_ret + [mkt_rf_label]
    st_mkt_cov = cb.loc[:, selected_cols].cov().iloc[:-1, -1].sum() # Covariance of stock returns & market returns
    st_ex_ret_var =  cb.filter(items = col_st_ex_ret, axis = 1).sum().var() # Variance of stock returns
    mkt_rf_var = cb[mkt_rf_label].var() # Variance of market returns
    st_mkt_corr = st_mkt_cov / (st_ex_ret_std * mkt_rf_std) # Correlation of stock and market
    
    pf_exp_ret, beta, alpha = 0, 0, 0
    for col in st_labels:
        y = cb[str(col + '.Exc.Return')].values # Indipendent variable of OLS
        x = sm.add_constant(cb[mkt_rf_label].values) # Didipendent variable of OLS
        model = sm.OLS(y, x)
        results = model.fit()
        beta += results.params[1]
        alpha += results.params[0]
    #cb['Exp.Ret'] = cb[RF_label] + beta * mkt_rf_avg    # Portfolio expected return
    #exp_ret = cb['Exp.Ret'].sum()
    
    output_stats = {
        'Start': cb.first_valid_index(),
        'End': cb.last_valid_index(),
        'Pf.Avg.Ret.%': st_ret_avg, 'Pf.Avg.Exc.Ret.%': st_ex_ret_avg,
        'Mkt.RF.Avg': mkt_rf_avg,
        'RF.Avg': RF_avg,
        'Pf.Sharpe-ratio': st_sharpe_ratio,
        'Alpha.%': alpha, 'Beta': beta,
        'Pf.Std.%': st_ex_ret_std, 'Mkt.RF.Std.%': mkt_rf_std,
        'Pf.Var': st_ex_ret_var, 'Mkt.RF.Var': mkt_rf_var,
        'Cov': st_mkt_cov,
        'Corr': st_mkt_corr,
        #'Expected.Ret.%': exp_ret,
    }
    
    #output_stats['St.PF.Value'] = balance
    #output_stats['Est.Act.PF.value'] = balance + (balance * (output_stats['Expected.Ret.%']))
    #output_stats['Est.Act.Profit'] = output_stats['Est.Act.PF.value'] - balance
    cb = cb.merge(stocks, left_index=True, right_index=True, how='inner', suffixes=('_Normalized', '_Real')) #Combining data togheter
    cb = cb.merge(mkt_rf, left_index=True, right_index=True, how='inner', suffixes=('_Normalized', '_Real'))
    df = pd.DataFrame(data = output_stats, index = [index])

    return df, cb.reindex(columns=sorted(cb.columns, reverse=True))

In [44]:
# Used in Point 7

def calculate_statistics_4f(pf, stocks, mf, mf_labels, rf, mkt_rf_label, index, start_date, end_date, balance):
    # Verifing weights and getting stocks label
    verify_weights(list(pf.values()))
    st_labels = list(pf.keys())
    
    # Adjusting input
    start_date = start_date.to_period('M')
    end_date = end_date.to_period('M')
    stocks_n = normalize_data(stocks.loc[start_date : end_date])
    
    # Creating label to access cb DataFrame
    col_st_ret = [col + '.Return' for col in st_labels]
    col_st_ex_ret = [col + '.Exc.Return' for col in st_labels]
    
    # cb is used to adjust data cardinality
    cb = stocks_n.merge(mf, left_index=True, right_index=True, how='inner') #Combining data togheter
    cb = cb.merge(rf, left_index=True, right_index=True, how='inner') #Combining data togheter

    # Calculating stock/s returns and excess return
    for column in st_labels:
            cb[column + '.Return'] = ((cb[column] / cb[column].shift(1)) - 1 ) * 100  * pf[column]
            cb[column + '.Exc.Return'] = cb[column + '.Return'] - (cb[mkt_rf_label]) * pf[column]
    with pd.option_context('mode.use_inf_as_na', True):   # Dropping columns with NaN or inf values
        cb.dropna(inplace=True)
    
    # Calculating average return for stock/s
    st_ret_avg = cb.filter(items = col_st_ret, axis = 1).sum(axis = 1).mean()
    fc_avg = cb[mkt_rf_label].mean()   # Average of factor

    # Calculating Average excess returns of stock and market
    st_ex_ret_avg = cb.filter(items = col_st_ex_ret, axis = 1).sum(axis = 1).mean() # Average stock excess return
    mkt_rf_avg = cb['Mkt-RF'].mean()

    # Calculating sharpe-ratio
    st_ex_ret_std = cb.filter(items = col_st_ex_ret, axis = 1).std().sum() # Standard dev. of stock excess return (Volatility)
    mf_std = cb[[col for col in mf_labels]].std().sum() # Standard dev. of market excess return (Volatility)
    st_sharpe_ratio = st_ex_ret_avg / st_ex_ret_std # Stock sharpe-ratio
    
    # Other statistics
    selected_cols = col_st_ex_ret + [col for col in mf_labels]
    st_mf_cov = cb.loc[:, selected_cols].cov().iloc[:-1, -1].sum()
    st_ex_ret_var =  cb.filter(items = col_st_ex_ret, axis = 1).sum().var() # Variance of stock returns
    mf_var = cb[[col for col in mf_labels]].var().sum() # Variance of market returns
    st_mf_corr = st_mf_cov / (st_ex_ret_std * mf_std) # Correlation of stock and market
    
    pf_exp_ret, beta, alpha = 0, 0, 0
    for col in st_labels:
        y = cb[str(col + '.Exc.Return')].values # Indipendent variable of OLS
        x = sm.add_constant(cb[[col for col in mf_labels]].values) # Didipendent variable of OLS
        model = sm.OLS(y, x)
        results = model.fit()
        beta += results.params[1]
        alpha += results.params[0]
   
    #cb['Exp.Ret'] = cb['RF'] + beta * mkt_rf_avg    # Portfolio expected return
    #exp_ret = cb['Exp.Ret'].sum()
    output_stats = {
        'Start': cb.first_valid_index(),
        'End': cb.last_valid_index(),
        'Pf.Avg.Ret.%': st_ret_avg, 'Pf.Avg.Exc.Ret.%': st_ex_ret_avg,
        'Mkt.RF.Avg': mkt_rf_avg,
        'RF.Avg': fc_avg,
        'Pf.Sharpe-ratio': st_sharpe_ratio,
        'Alpha.%': alpha, 'Beta': beta,
        'Pf.Std.%': st_ex_ret_std, 'Mf.Std.%': mf_std,
        'Pf.Var': st_ex_ret_var, 'Mf.Var': mf_var,
        'Cov': st_mf_cov,
        'Corr': st_mf_corr,
        #'Expected.Ret.%': exp_ret,
    }
    
    #output_stats['St.PF.Value'] = balance
    #output_stats['Est.Act.PF.value'] = balance + (balance * (output_stats['Expected.Ret.%']))
    #output_stats['Est.Act.Profit'] = output_stats['Est.Act.PF.value'] - balance
    cb = cb.merge(stocks, left_index=True, right_index=True, how='inner', suffixes=('_Normalized', '_Real')) #Combining data togheter
    #cb = cb.merge(mkt_rf, left_index=True, right_index=True, how='inner', suffixes=('_Normalized', '_Real'))
    df = pd.DataFrame(data = output_stats, index = [index])

    return df, cb.reindex(columns=sorted(cb.columns, reverse=True))

In [ ]:
# Used in Point 8

def calculate_statistics_5f(pf, stocks, future, future_label, mkt_rf, mkt_rf_label, index, balance, start_date, end_date):
    
    # Input operations
    verify_weights(list(pf.values()))   # Verifing weights and getting stocks label
    st_labels = list(pf.keys())
    col_st_ret = [col + '.Return' for col in st_labels]   # Creating label to access combined_datas(cb) DataFrame
    ft_ret_label = future_label + '.Return'
    start_date = start_date.to_period('M')   # Adjusting input parameters
    end_date = end_date.to_period('M')
    stocks_n = normalize_data(stocks.loc[start_date : end_date, :])  
    future_n = normalize_data(future.loc[start_date : end_date, :]) 
    #------------------------------------------------------------
    cb = stocks_n.merge(future_n, left_index=True, right_index=True, how='inner')   # Join all dataset for calculation (Cardinality and forced dateperiod match)
    cb = cb.merge(mkt_rf, left_index=True, right_index=True, how='inner')   # Join all dataset for calculation (Cardinality and forced dateperiod match)

    # Calculating stock/s returns and excess returns
    for column in [col for col in st_labels + [future_label]]:
        if column != future_label:
            cb[column + '.Return'] = ((cb[column] / cb[column].shift(1)) - 1 ) * 100 * pf[column]
        else:
            cb[column + '.Return'] = ((cb[column] / cb[column].shift(1)) - 1) * 100
    with pd.option_context('mode.use_inf_as_na', True):
        cb.dropna(inplace=True)

    st_ret_avg = cb.filter(items = col_st_ret, axis = 1).sum(axis = 1).mean()   # Calculating average return for stocks
    ft_ret_avg = cb[future_label].mean()
    fc_avg = cb[mkt_rf_label].mean()

    ft_pf_ret = ((future.iloc[-1][future_label] - future.iloc[0][future_label]) / future.iloc[0][future_label])
    
    # Calculating Alpha and Beta 
    pf_exp_ret, opt_hedge_ratio, alpha = 0.0, 0.0, 0.0
    for col in st_labels:
        y = cb[str(col + '.Return')].values # Dipendent variable of OLS
        x = sm.add_constant(cb[ft_ret_label].values) # Indipendent variable of OLS
        model = sm.OLS(y, x)
        results = model.fit()
        opt_hedge_ratio += results.params[1]
        
    pf_value = get_portfolio_value(pf, balance, stocks)
    ft_value = future.loc[start_date : end_date, :].mean().mean() * 50
    req_ft_num = round(opt_hedge_ratio * (pf_value / (ft_value)))
    
    # Output operations------------------------------------------
    output_stats = {
        'Start': cb.first_valid_index(),
        'End': cb.last_valid_index(),
        'Pf.Avg.Ret.%': st_ret_avg, 'Ft.Avg.Ret.%': ft_ret_avg,
        'RF.Avg': fc_avg,
        'Future.Ret.': ft_pf_ret,
        'Pf.Value': pf_value,
        'Future.value': ft_value,
        'Hedge-Ratio': opt_hedge_ratio,
        'Futures.Needed': req_ft_num
    }
    
    cb = cb.merge(stocks, left_index=True, right_index=True, how='inner', suffixes=('_Normalized', '_Real')) #Combining data togheter
    cb = cb.merge(future, left_index=True, right_index=True, how='inner', suffixes=('_Normalized', '_Real'))
    df = pd.DataFrame(data = output_stats, index = [index])
    return df, cb.reindex(columns=sorted(cb.columns, reverse=True))

## Points

### Common settings

In [49]:
# Date Parameters
start_date_30 = pd.to_datetime('1993-01-01') # Period 1 - 30 years
start_date_20 = pd.to_datetime('2003-01-01') # Period 2 - 20 years 
start_date_15 = pd.to_datetime('2008-01-01') # Period 3 - 15 years
start_date_10 = pd.to_datetime('2013-01-01') # Period 4 - 10 years 
start_date_5 = pd.to_datetime('2017-12-01') # Period 5 - 5 years 
end_date = pd.to_datetime('2023-01-01')

# Custom periods - 2008/2009 crisis (c1) 
start_date_crisis = pd.to_datetime('2008-07-01')
end_date_crisis = pd.to_datetime('2009-03-01')

balance = 1000000

### **POINT 1** - Single stock analysis
Collect price information of ‘your stock’ and stock market (S&P 500 index) from Yahoo Finance for the past 5 years, 15 years, and 30 years. 
Note that you have price information – not returns – calculate returns  before you do anything since the CAPM is all about stock returns. 
Using Python calculate parts i) through iv) below (Risk-free returns can be downloaded at http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html)
Comment on your results.
- 1 Average Excess returns (excess returns are defined as “returns of a stock/portfolio minus risk-free returns)
- 2 Sharpe-ratio, which is Mean/Standard deviation of excess returns.
- 3 Alpha, which is the constant term in Ordinary Least Squares Regression (OLS). 
- 4 Beta, which is the coefficient of the explanatory variable. 
- 5 Comment on whether your estimates are correct (check for ‘beta’ that you can find in Yahoo Finance to compare).

In [52]:
# Portfolio settings
market_label = '^GSPC'
factor_label = ['RF']
portfolio = {'PG': 1}

print("------------------------------------------------------------------------------------------\n")
print("PORTFOLIO\nPortfolio composition  -> ", portfolio, "\nMarket index ->", market_label, "\nFama/French 3 rate-> ", factor_label, "\n")

# Retrieving all data needed
market, stocks, factor = get_datas(portfolio, market_label, factor_label, start_date_30, end_date)

# Calculating statistics
pf_stats_30, cb_30 = calculate_statistics_1f(portfolio, stocks, market, market_label, factor, 'RF', 'Last 30Y', start_date_30, end_date, balance) 
pf_stats_15, cb_15  = calculate_statistics_1f(portfolio, stocks, market, market_label, factor, 'RF', 'Last 15Y', start_date_15, end_date, balance)
pf_stats_5, cb_5   = calculate_statistics_1f(portfolio, stocks, market, market_label, factor, 'RF', 'Last 5Y', start_date_5, end_date, balance)
pf_stats = pd.concat([pf_stats_5, pf_stats_15, pf_stats_30], axis = 0)

# Printing output - If data not show properly, try to resize the page (Ex. ctrl + mouseWheel, ctrl + '+'/'-')
profit_lb = 'Profit on ' + str(balance) + '$'
print_results(pf_stats)
columns=[col for col in cb_30.columns if col.endswith('.Return') and not col.endswith('.Exc.Return')]
chart(cb_30.filter(regex='_Normalized$'), cb_30.filter(items = columns), "Prices chart", "Returns chart", "Charts section")
#print_datas(cb_30)

------------------------------------------------------------------------------------------

PORTFOLIO
Portfolio composition  ->  {'PG': 1} 
Market index -> ^GSPC 
Fama/French 3 rate->  ['RF'] 

-> Starting downloading market and portfolio's components data ...
-> Download Complete.


Estimations for the portfolio: 

┌──────────┬─────────┬─────────┬────────────────┬────────────────┬────────────────────┬────────────────────┬──────────┬───────────────────┬───────────┬────────┬────────────┬────────────┬──────────┬──────────┬─────────┬────────┐
│          │ Start   │ End     │   Pf.Avg.Ret.% │   Mk.Avg.Ret.% │   Pf.Avg.Exc.Ret.% │   Mk.Avg.Exc.Ret.% │   RF.Avg │   Pf.Sharpe-ratio │   Alpha.% │   Beta │   Pf.Std.% │   Mk.Std.% │   Pf.Var │   Mk.Var │     Cov │   Corr │
├──────────┼─────────┼─────────┼────────────────┼────────────────┼────────────────────┼────────────────────┼──────────┼───────────────────┼───────────┼────────┼────────────┼────────────┼──────────┼──────────┼─────────┼────────

/tmp/ipykernel_23949/377196534.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  norm_data[column] = norm_data[column] / norm_data[column][0]
/tmp/ipykernel_23949/377196534.py:12: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/tmp/ipykernel_23949/377196534.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  norm_data[column] = norm_data[column] / norm_data[column][0]
/tmp/ipykernel_23949/377196534.py:12: FutureWarning: use_inf_as_na option is deprecated and will be

### **POINT 2** 

Now consider that you are a portfolio manager and you have $1 million to invest, and hence you want to diversify the risk of owning ‘your stock’.

Pick any number of stocks (10 minimum) you want to create an equally weighted stock portfolio of stocks and calculate for the portfolio:
- 1 Average Excess returns (excess returns are defined as “returns of a stock/portfolio minus risk-free returns);
- 2 Sharpe-ratio: which is Mean/Standard deviation of excess returns;
- 3 Alpha, which is the constant term in Ordinary Least Squares Regression (OLS);
- 4 Beta, which is the coefficient of the explanatory variable;

Comment on your results.


In [53]:
# Settings
market_label = '^GSPC'
factor_label = ['RF']
#portfolio = {'NKE':0.1, 'JNJ':0.1, 'MCD':0.1, 'IBM':0.1, 'INTC':0.1, 'GE':0.1, 'F':0.1, 'AXP':0.1, 'KO':0.1, 'CL':0.1} 
portfolio = {'MSFT':0.1, 'INTC':0.1, 'JNJ':0.1, 'PFE':0.1, 'XOM':0.1, 'AXP':0.1, 'DIS':0.1, 'KO':0.1, 'BA':0.1, 'PG':0.1} # BA: Boeing, PG:procter&gambe company

#portfolio = {'FDX':1} 
    
print("------------------------------------------------------------------------------------------\n")
print("PORTFOLIO\nPortfolio composition  -> ", portfolio, "\nMarket index ->", market_label, "\nFama/French 3 rate-> ", factor_label, "\n")

market, stocks, factor = get_datas(portfolio, market_label, factor_label, start_date_30, end_date)

# Calculating statistics # Point 2, 3
pf_stats_30, cb_30 = calculate_statistics_2f(portfolio, stocks, market, market_label, factor, 'RF', 'Last 30Y', start_date_30, end_date, balance) 
pf_stats_20, cb_20  = calculate_statistics_2f(portfolio, stocks, market, market_label, factor, 'RF', 'Last 20Y', start_date_20, end_date, balance)
pf_stats_10, cb_10  = calculate_statistics_2f(portfolio, stocks, market, market_label, factor, 'RF', 'Last 10Y', start_date_10, end_date, balance)
pf_stats = pd.concat([pf_stats_10, pf_stats_20, pf_stats_30], axis = 0)

# Printing output - If data not show properly, try to resize the page (Ex. ctrl + mouseWheel, ctrl + '+'/'-')
print_results(pf_stats)
columns=[col for col in cb_30.columns if col.endswith('.Return') and not col.endswith('.Exc.Return')]
chart(cb_30.filter(regex='_Normalized$'), cb_30.filter(items = columns), "Prices chart", "Returns chart", "Charts section")
#print_datas(cb_30)

------------------------------------------------------------------------------------------

PORTFOLIO
Portfolio composition  ->  {'MSFT': 0.1, 'INTC': 0.1, 'JNJ': 0.1, 'PFE': 0.1, 'XOM': 0.1, 'AXP': 0.1, 'DIS': 0.1, 'KO': 0.1, 'BA': 0.1, 'PG': 0.1} 
Market index -> ^GSPC 
Fama/French 3 rate->  ['RF'] 

-> Starting downloading market and portfolio's components data ...
-> Download Complete.


Estimations for the portfolio: 

┌──────────┬─────────┬─────────┬────────────────┬────────────────┬────────────────────┬────────────────────┬──────────┬───────────────────┬───────────┬────────┬────────────┬────────────┬──────────┬──────────┬─────────┬────────┐
│          │ Start   │ End     │   Pf.Avg.Ret.% │   Mk.Avg.Ret.% │   Pf.Avg.Exc.Ret.% │   Mk.Avg.Exc.Ret.% │   RF.Avg │   Pf.Sharpe-ratio │   Alpha.% │   Beta │   Pf.Std.% │   Mk.Std.% │   Pf.Var │   Mk.Var │     Cov │   Corr │
├──────────┼─────────┼─────────┼────────────────┼────────────────┼────────────────────┼────────────────────┼────────

/tmp/ipykernel_23949/377196534.py:19: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_23949/377196534.py:12: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.

/tmp/ipykernel_23949/377196534.py:19: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_23949/377196534.py:12: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.

/tmp/ipykernel_23949/377196534.py:19: FutureWarning:

Series.__getitem__ treating keys as 

### **POINT 3** 

Portfolio pefcormance during the recent crisis – August 2008 through March 2009


In [54]:
# Settings
market_label = '^GSPC'
factor_label = ['RF']
#portfolio = {'AAPL':0.1, 'GOOGL':0.1, 'JNJ':0.1, 'ABT':0.1, 'BAC':0.1, 'AXP':0.1, 'NKE':0.1, 'KO':0.1, 'PEP':0.1, 'BA':0.1}
#portfolio = {'NKE':0.1, 'JNJ':0.1, 'MCD':0.1, 'IBM':0.1, 'INTC':0.1, 'GE':0.1, 'F':0.1, 'AXP':0.1, 'KO':0.1, 'CL':0.1} 
portfolio = {'MSFT':0.1, 'INTC':0.1, 'JNJ':0.1, 'PFE':0.1, 'XOM':0.1, 'AXP':0.1, 'DIS':0.1, 'KO':0.1, 'BA':0.1, 'PG':0.1}

print("------------------------------------------------------------------------------------------\n")
print("PORTFOLIO\nPortfolio composition  -> ", portfolio, "\nMarket index ->", market_label, "\nFama/French 3 rate-> ", factor_label, "\n")

market, stocks, factor = get_datas(portfolio, market_label, factor_label, start_date_30, end_date)

# Calculating statistics # Point 2, 3
#pf_stats_30, cb_30 = calculate_statistics_2f(portfolio, stocks, market, market_label, factor, 'RF', 'Last 30Y', start_date_30, end_date, balance) 
#pf_stats_20, cb_20  = calculate_statistics_2f(portfolio, stocks, market, market_label, factor, 'RF', 'Last 20Y', start_date_20, end_date, balance)
#pf_stats_10, cb_10  = calculate_statistics_2f(portfolio, stocks, market, market_label, factor, 'RF', 'Last 10Y', start_date_10, end_date, balance)
pf_stats_crisis, cb_crisis  = calculate_statistics_2f(portfolio, stocks, market, market_label, factor, 'RF', '08/09 Crisis', start_date_crisis, end_date_crisis, balance)
pf_stats = pd.concat([pf_stats_crisis], axis = 0)

#Printing output - If data not show properly, try to resize the page (Ex. ctrl + mouseWheel, ctrl + '+'/'-')
print_results(pf_stats)

columns=[col for col in cb_30.columns if col.endswith('.Return') and not col.endswith('.Exc.Return')]
chart(cb_30.filter(regex='_Normalized$'), cb_30.filter(items = columns), "Prices chart", "Returns chart", "Charts section")
#print_datas(cb_30)

------------------------------------------------------------------------------------------

PORTFOLIO
Portfolio composition  ->  {'MSFT': 0.1, 'INTC': 0.1, 'JNJ': 0.1, 'PFE': 0.1, 'XOM': 0.1, 'AXP': 0.1, 'DIS': 0.1, 'KO': 0.1, 'BA': 0.1, 'PG': 0.1} 
Market index -> ^GSPC 
Fama/French 3 rate->  ['RF'] 

-> Starting downloading market and portfolio's components data ...
-> Download Complete.


Estimations for the portfolio: 

┌──────────────┬─────────┬─────────┬────────────────┬────────────────┬────────────────────┬────────────────────┬──────────┬───────────────────┬───────────┬────────┬────────────┬────────────┬──────────┬──────────┬─────────┬────────┐
│              │ Start   │ End     │   Pf.Avg.Ret.% │   Mk.Avg.Ret.% │   Pf.Avg.Exc.Ret.% │   Mk.Avg.Exc.Ret.% │   RF.Avg │   Pf.Sharpe-ratio │   Alpha.% │   Beta │   Pf.Std.% │   Mk.Std.% │   Pf.Var │   Mk.Var │     Cov │   Corr │
├──────────────┼─────────┼─────────┼────────────────┼────────────────┼────────────────────┼─────────────────

/tmp/ipykernel_23949/377196534.py:19: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_23949/377196534.py:12: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.



### **POINT 4** 
Now add a risk-free bond to your portfolio. 
Assume that you are a risk-averse manager, and you want 40% of $1 million invested in risk-free rates and the rest (60%) in the stock portfolio you created in Point 3. 

Did the portfolio pefcorm better than the “stocks” only portfolio during the 2007-2009 crisis ?


In [55]:
# Settings
market_label = '^GSPC'
risk_free_bond = '^TNX'
factor_label = ['RF']

#portfolio_1 = {'NKE':0.1, 'JNJ':0.1, 'MCD':0.1, 'IBM':0.1, 'INTC':0.1, 'GE':0.1, 'F':0.1, 'AXP':0.1, 'KO':0.1, 'CL':0.1} 
#portfolio_2 = {risk_free_bond: 0.4, 'NKE':0.06, 'MCD':0.06, 'IBM':0.06, 'INTC':0.06, 'GE':0.06, 'F':0.06, 'JNJ':0.06, 'AXP':0.06, 'KO':0.06, 'CL':0.06}

portfolio_1 = {'MSFT':0.1, 'INTC':0.1, 'JNJ':0.1, 'PFE':0.1, 'XOM':0.1, 'AXP':0.1, 'DIS':0.1, 'KO':0.1, 'BA':0.1, 'PG':0.1}
portfolio_2 = {risk_free_bond: 0.4,'MSFT':0.06, 'INTC':0.06, 'JNJ':0.06, 'PFE':0.06, 'XOM':0.06, 'AXP':0.06, 'DIS':0.06, 'KO':0.06, 'BA':0.06, 'PG':0.06}

print("------------------------------------------------------------------------------------------\n")
print("PORTFOLIO Only Stocks\n")
print("Portfolio composition  -> ", portfolio_1, "\nMarket index ->", market_label, "\nFama/French 3 rate-> ", factor_label, "\n")

market, stocks, factor = get_datas(portfolio_1, market_label, factor_label, start_date_30, end_date)

# Calculating statistics for portfolio with only stocks
#pf_stats_30, cb_30_1 = calculate_statistics_2f(portfolio_1, stocks, market, market_label, factor, 'RF', 'Last 30Y', start_date_30, end_date, balance) 
#pf_stats_20, cb_20_1  = calculate_statistics_2f(portfolio_1, stocks, market, market_label, factor, 'RF', 'Last 20Y', start_date_20, end_date, balance)
#pf_stats_10, cb_10_1  = calculate_statistics_2f(portfolio_1, stocks, market, market_label, factor, 'RF', 'Last 10Y', start_date_10, end_date, balance)

pf_stats_crisis, cb_crisis_1  = calculate_statistics_2f(portfolio_1, stocks, market, market_label, factor, 'RF', '08/09 Crisis', start_date_crisis, end_date_crisis, balance)
pf_stats_1 = pd.concat([pf_stats_crisis], axis = 0)

print_results(pf_stats_1)

print("PORTFOLIO with Bond\n")
print("Portfolio composition  -> ", portfolio_2, "\nMarket index ->", market_label, "\nFama/French 3 rate-> ", factor_label, "\n")

market, stocks, factor = get_datas(portfolio_2, market_label, factor_label, start_date_30, end_date)

# Calculating statistics for portfolio reorganized with a bond
#pf_stats_30, cb_30_2 = calculate_statistics_2f(portfolio_2, stocks, market, market_label, factor, 'RF', 'Last 30Y', start_date_30, end_date, balance) 
#pf_stats_20, cb_20_2  = calculate_statistics_2f(portfolio_2, stocks, market, market_label, factor, 'RF', 'Last 20Y', start_date_20, end_date, balance)
#pf_stats_10, cb_10_2 = calculate_statistics_2f(portfolio_2, stocks, market, market_label, factor, 'RF', 'Last 10Y', start_date_10, end_date, balance)

pf_stats_crisis, cb_crisis_2  = calculate_statistics_2f(portfolio_2, stocks, market, market_label, factor, 'RF', '08/09 Crisis', start_date_crisis, end_date_crisis, balance)
pf_stats_2 = pd.concat([pf_stats_crisis], axis = 0)

# Printing output - If data not show properly, try to resize the page (Ex. ctrl + mouseWheel, ctrl + '+'/'-')
print_results(pf_stats_2)
columns=[col for col in cb_crisis_1.columns if col.endswith('.Return') and not col.endswith('.Exc.Return')]
chart(cb_crisis_1.filter(regex='_Normalized$'), cb_crisis_1.filter(items = columns), "Prices chart", "Returns chart", "Charts section")
chart(cb_crisis_2.filter(regex='_Normalized$'), cb_crisis_2.filter(items = columns), "Prices chart", "Returns chart", "Charts section")

#print_datas(cb_crisis_1)
#print_datas(cb_crisis_2)


------------------------------------------------------------------------------------------

PORTFOLIO Only Stocks

Portfolio composition  ->  {'MSFT': 0.1, 'INTC': 0.1, 'JNJ': 0.1, 'PFE': 0.1, 'XOM': 0.1, 'AXP': 0.1, 'DIS': 0.1, 'KO': 0.1, 'BA': 0.1, 'PG': 0.1} 
Market index -> ^GSPC 
Fama/French 3 rate->  ['RF'] 

-> Starting downloading market and portfolio's components data ...
-> Download Complete.


Estimations for the portfolio: 

┌──────────────┬─────────┬─────────┬────────────────┬────────────────┬────────────────────┬────────────────────┬──────────┬───────────────────┬───────────┬────────┬────────────┬────────────┬──────────┬──────────┬─────────┬────────┐
│              │ Start   │ End     │   Pf.Avg.Ret.% │   Mk.Avg.Ret.% │   Pf.Avg.Exc.Ret.% │   Mk.Avg.Exc.Ret.% │   RF.Avg │   Pf.Sharpe-ratio │   Alpha.% │   Beta │   Pf.Std.% │   Mk.Std.% │   Pf.Var │   Mk.Var │     Cov │   Corr │
├──────────────┼─────────┼─────────┼────────────────┼────────────────┼────────────────────┼────

/tmp/ipykernel_23949/377196534.py:19: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_23949/377196534.py:12: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.



-> Download Complete.


Estimations for the portfolio: 

┌──────────────┬─────────┬─────────┬────────────────┬────────────────┬────────────────────┬────────────────────┬──────────┬───────────────────┬───────────┬────────┬────────────┬────────────┬──────────┬──────────┬─────────┬────────┐
│              │ Start   │ End     │   Pf.Avg.Ret.% │   Mk.Avg.Ret.% │   Pf.Avg.Exc.Ret.% │   Mk.Avg.Exc.Ret.% │   RF.Avg │   Pf.Sharpe-ratio │   Alpha.% │   Beta │   Pf.Std.% │   Mk.Std.% │   Pf.Var │   Mk.Var │     Cov │   Corr │
├──────────────┼─────────┼─────────┼────────────────┼────────────────┼────────────────────┼────────────────────┼──────────┼───────────────────┼───────────┼────────┼────────────┼────────────┼──────────┼──────────┼─────────┼────────┤
│ 08/09 Crisis │ 2008-08 │ 2009-03 │        -3.8508 │        -5.3155 │            -3.9033 │            -5.3680 │   0.0525 │           -0.3062 │   -3.0055 │ 0.1673 │    12.7476 │     8.1917 │   9.2721 │  67.1044 │ 11.2240 │ 0.1075 │
└──────────────

/tmp/ipykernel_23949/377196534.py:19: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_23949/377196534.py:12: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.




 Charts section


### **POINT 5** 

Use the Mkt-rf (not the market risk premium you have calculated in step 1) given by those authors and redo Question Point 2.

In [56]:
# Settings
market_label = '^GSPC'
factor_label = ['RF']
mkt_factor_label = ['Mkt-RF']
#portfolio = {'NKE':0.1, 'JNJ':0.1, 'MCD':0.1, 'IBM':0.1, 'INTC':0.1, 'GE':0.1, 'F':0.1, 'AXP':0.1, 'KO':0.1, 'CL':0.1} 
portfolio = {'MSFT':0.1, 'INTC':0.1, 'JNJ':0.1, 'PFE':0.1, 'XOM':0.1, 'AXP':0.1, 'DIS':0.1, 'KO':0.1, 'BA':0.1, 'PG':0.1}

print("------------------------------------------------------------------------------------------\n")
print("PORTFOLIO analysis for RF and Mkt-RF factors\n")
print("Portfolio composition  -> ", portfolio, "\nMarket index ->", market_label, "\n")

market, stocks, factor = get_datas(portfolio, market_label, factor_label, start_date_30, end_date)
mkt_rf = get_fama_french_3(mkt_factor_label)

# Calculating statistics - Like point 2
pf_stats_30, cb_30_1 = calculate_statistics_2f(portfolio, stocks, market, market_label, factor, 'RF', 'Last 30Y', start_date_30, end_date, balance) 
pf_stats_20, cb_20_1  = calculate_statistics_2f(portfolio, stocks, market, market_label, factor, 'RF', 'Last 20Y', start_date_20, end_date, balance)
pf_stats_10, cb_10_1  = calculate_statistics_2f(portfolio, stocks, market, market_label, factor, 'RF', 'Last 10Y', start_date_10, end_date, balance)
#pf_stats_crisis, cb_crisis_1  = calculate_statistics_2f(portfolio, stocks, market, market_label, factor, 'RF', '08/09 Crisis', start_date_crisis, end_date_crisis, balance)
pf_stats_1 = pd.concat([pf_stats_10, pf_stats_20, pf_stats_30], axis = 0)

pf_stats_30, cb_30_2 = calculate_statistics_3f(portfolio, stocks, mkt_rf, 'Mkt-RF', factor, 'RF', 'Last 30Y', start_date_30, end_date, balance) 
pf_stats_20, cb_20_2 = calculate_statistics_3f(portfolio, stocks, mkt_rf, 'Mkt-RF', factor, 'RF', 'Last 20Y', start_date_20, end_date, balance)
pf_stats_10, cb_10_2 = calculate_statistics_3f(portfolio, stocks, mkt_rf, 'Mkt-RF', factor, 'RF', 'Last 10Y', start_date_10, end_date, balance)
#pf_stats_crisis, cb_crisis_2 = calculate_statistics_3f(portfolio, stocks, mkt_rf, 'Mkt-RF', factor, 'RF', '08/09 Crisis', start_date_crisis, end_date_crisis, balance)
pf_stats_2 = pd.concat([pf_stats_10, pf_stats_20, pf_stats_30], axis = 0)

# Printing output - If data not show properly, try to resize the page (Ex. ctrl + mouseWheel, ctrl + '+'/'-')
print_results(pf_stats_1, str("Fama/French 3 rate-> RF (Like point 2)"))
print_results(pf_stats_2, str("Fama/French 3 rate-> Mkt-RF"))
columns=[col for col in cb_30.columns if col.endswith('.Return') and not col.endswith('.Exc.Return')]
chart(cb_30_1.filter(regex='_Normalized$'), cb_30_1.filter(items = columns), "Prices chart", "Returns chart", "Charts section (RF)")
chart(cb_30_2.filter(regex='_Normalized$'), cb_30_2.filter(items = columns), "Prices chart", "Returns chart", "Charts section (Mkt-RF)")
#print_datas(cb_30_1)
#print_datas(cb_30_2)

------------------------------------------------------------------------------------------

PORTFOLIO analysis for RF and Mkt-RF factors

Portfolio composition  ->  {'MSFT': 0.1, 'INTC': 0.1, 'JNJ': 0.1, 'PFE': 0.1, 'XOM': 0.1, 'AXP': 0.1, 'DIS': 0.1, 'KO': 0.1, 'BA': 0.1, 'PG': 0.1} 
Market index -> ^GSPC 

-> Starting downloading market and portfolio's components data ...
-> Download Complete.

Fama/French 3 rate-> RF (Like point 2)
┌──────────┬─────────┬─────────┬────────────────┬────────────────┬────────────────────┬────────────────────┬──────────┬───────────────────┬───────────┬────────┬────────────┬────────────┬──────────┬──────────┬─────────┬────────┐
│          │ Start   │ End     │   Pf.Avg.Ret.% │   Mk.Avg.Ret.% │   Pf.Avg.Exc.Ret.% │   Mk.Avg.Exc.Ret.% │   RF.Avg │   Pf.Sharpe-ratio │   Alpha.% │   Beta │   Pf.Std.% │   Mk.Std.% │   Pf.Var │   Mk.Var │     Cov │   Corr │
├──────────┼─────────┼─────────┼────────────────┼────────────────┼────────────────────┼──────────────────

/tmp/ipykernel_23949/377196534.py:19: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_23949/377196534.py:12: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.

/tmp/ipykernel_23949/377196534.py:19: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_23949/377196534.py:12: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.

/tmp/ipykernel_23949/377196534.py:19: FutureWarning:

Series.__getitem__ treating keys as 


 Charts section (Mkt-RF)


### **POINT 7** 

Fama-French talks about different factors for stock/asset returns. 
Pleases run a multifactor model for the stock and portfolio you have created. 

The factors are: 
- Market;
- SMB;
- HML;
- TERM=difference in yields between 10 year and 3 months U.S. Treasuries;
- CREDIT=difference in yields between BAA and AAA rated U.S. corporate bonds. 

The factors can be found/created from the French/U.S. Federal Reserve website data. 

In [57]:
# Settings
market_label = '^GSPC'
factor_labels = ['Mkt-RF', 'SMB', 'HML']
mkt_factor_label = 'Mkt-RF'
rf_label = ['RF']
#portfolio = {'NKE':0.1, 'JNJ':0.1, 'MCD':0.1, 'IBM':0.1, 'INTC':0.1, 'GE':0.1, 'F':0.1, 'AXP':0.1, 'KO':0.1, 'CL':0.1} 
portfolio = {'MSFT':0.1, 'INTC':0.1, 'JNJ':0.1, 'PFE':0.1, 'XOM':0.1, 'AXP':0.1, 'DIS':0.1, 'KO':0.1, 'BA':0.1, 'PG':0.1}

print("------------------------------------------------------------------------------------------\n")
print("PORTFOLIO multifactor model\nPortfolio composition  -> ", portfolio, "\nMarket index ->", market_label, "\n")

market, stocks, factors = get_datas(portfolio, market_label, factor_labels, start_date_30, end_date)
rf = get_fama_french_3(rf_label)
term = term_factor(start_date_30, end_date, 'TERM')
credit = credit_factor(start_date_30, end_date)

factors = factors.merge(term, left_index=True, right_index=True, how='inner')
factors = factors.merge(credit, left_index=True, right_index=True, how='inner')
factor_labels.append('TERM')
factor_labels.append('CREDIT')

pf_stats_30, cb_30 = calculate_statistics_4f(portfolio, stocks, factors, factor_labels, rf, 'Mkt-RF', 'Last 30Y', start_date_30, end_date, balance) 
pf_stats_20, cb_20  = calculate_statistics_4f(portfolio, stocks, factors, factor_labels, rf, 'Mkt-RF', 'Last 20Y', start_date_20, end_date, balance)
pf_stats_10, cb_10  = calculate_statistics_4f(portfolio, stocks, factors, factor_labels, rf, 'Mkt-RF', 'Last 10Y', start_date_10, end_date, balance)
#pf_stats_crisis, cb_crisis  = calculate_statistics_4f(portfolio, stocks, factors, factor_labels, rf, 'Mkt-RF', '08/09 Crisis', start_date_crisis, end_date_crisis, balance)
pf_stats = pd.concat([pf_stats_10, pf_stats_20, pf_stats_30], axis = 0)

# Printing output - If data not show properly, try to resize the page (Ex. ctrl + mouseWheel, ctrl + '+'/'-')
print_results(pf_stats, str("Portfolio multifactor model. Factors -> " + str(factor_labels)))
columns=[col for col in cb_30.columns if col.endswith('.Return') and not col.endswith('.Exc.Return')]
chart(cb_30.filter(regex='_Normalized$'), cb_30.filter(items = columns), "Prices chart", "Returns chart", "Charts section")
#print_datas(cb_30)
chart(term,cb_30.filter(items = columns),'Term','','')

------------------------------------------------------------------------------------------

PORTFOLIO multifactor model
Portfolio composition  ->  {'MSFT': 0.1, 'INTC': 0.1, 'JNJ': 0.1, 'PFE': 0.1, 'XOM': 0.1, 'AXP': 0.1, 'DIS': 0.1, 'KO': 0.1, 'BA': 0.1, 'PG': 0.1} 
Market index -> ^GSPC 

-> Starting downloading market and portfolio's components data ...
-> Download Complete.

Portfolio multifactor model. Factors -> ['Mkt-RF', 'SMB', 'HML', 'TERM', 'CREDIT']
┌──────────┬─────────┬─────────┬────────────────┬────────────────────┬──────────────┬──────────┬───────────────────┬───────────┬─────────┬────────────┬────────────┬──────────┬──────────┬─────────┬─────────┐
│          │ Start   │ End     │   Pf.Avg.Ret.% │   Pf.Avg.Exc.Ret.% │   Mkt.RF.Avg │   RF.Avg │   Pf.Sharpe-ratio │   Alpha.% │    Beta │   Pf.Std.% │   Mf.Std.% │   Pf.Var │   Mf.Var │     Cov │    Corr │
├──────────┼─────────┼─────────┼────────────────┼────────────────────┼──────────────┼──────────┼───────────────────┼─────

/tmp/ipykernel_23949/377196534.py:19: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_23949/3093263374.py:25: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.

/tmp/ipykernel_23949/377196534.py:19: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_23949/3093263374.py:25: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.

/tmp/ipykernel_23949/377196534.py:19: FutureWarning:

Series.__getitem__ treating keys a

### **POINT 8** 

Assume, there is only one factor, the market factor. 
You’d like to hedge the systematic risk/market risk in your portfolio since you expect an economic downturn. 

One way to achieve the goal is to use futures contracts. Specifically, you may want to use S&P500 futures to hedge. 

To accomplish this, you need to calculate the optimal hedge ratio since your portfolio may be related to the S&P500 futures differently than it is related to the S&P500 index, and then find # of futures and establish a strategy.
How many futures do you need and is your hedging strategy?

Steps to hedge your portfolio:
- Download monthly S&P500 futures data from Investing.com website http://www.investing.com/indices/us-spx-500-futures-historical-data
- Run a regression where the dependent variable is your portfolio returns and the explanatory variable is %change in futures prices; the coefficient of the explanatory variable is the “optimal hedge ratio”.
- Find the number of futures needed, which is “optimal hedge ratio”*(portfolio value/value of one S&P500 futures).


In [60]:
# Settings
future_label = 'ES2306=F'
future_label_2 = '^GSPC' # This label is also used to rapresent future
factor_label = ['Mkt-RF']
#portfolio = {'NKE':0.1, 'JNJ':0.1, 'MCD':0.1, 'IBM':0.1, 'INTC':0.1, 'GE':0.1, 'F':0.1, 'AXP':0.1, 'KO':0.1, 'CL':0.1} ù
portfolio = {'MSFT':0.1, 'INTC':0.1, 'JNJ':0.1, 'PFE':0.1, 'XOM':0.1, 'AXP':0.1, 'DIS':0.1, 'KO':0.1, 'BA':0.1, 'PG':0.1}

print("------------------------------------------------------------------------------------------\n")
print("PORTFOLIO\nPortfolio composition  -> ", portfolio, "\nFuture index ->", future_label, "\nFama/French 3 rate-> ", factor_label, "\n")

future, stocks, factor = get_datas(portfolio, future_label, factor_label, start_date_30, end_date)

# Calculating statistics # Point 2, 3
pf_stats_30, cb_30 = calculate_statistics_5f(portfolio, stocks, future, future_label_2, factor, 'Mkt-RF', 'Last 30Y',  balance, start_date_30, end_date) 
pf_stats_20, cb_20  = calculate_statistics_5f(portfolio, stocks, future, future_label_2, factor, 'Mkt-RF',  'Last 20Y', balance, start_date_20, end_date)
pf_stats_10, cb_10  = calculate_statistics_5f(portfolio, stocks, future, future_label_2, factor, 'Mkt-RF',  'Last 10Y', balance, start_date_10, end_date)
pf_stats = pd.concat([pf_stats_10, pf_stats_20, pf_stats_30], axis = 0)

#Printing output - If data not show properly, try to resize the page (Ex. ctrl + mouseWheel, ctrl + '+'/'-')
print_results(pf_stats)
columns=[col for col in cb_30.columns if col.endswith('.Return') and not col.endswith('.Exc.Return')]
#print_datas(cb_30)

------------------------------------------------------------------------------------------

PORTFOLIO
Portfolio composition  ->  {'MSFT': 0.1, 'INTC': 0.1, 'JNJ': 0.1, 'PFE': 0.1, 'XOM': 0.1, 'AXP': 0.1, 'DIS': 0.1, 'KO': 0.1, 'BA': 0.1, 'PG': 0.1} 
Future index -> ES2306=F 
Fama/French 3 rate->  ['Mkt-RF'] 

-> Starting downloading market and portfolio's components data ...



1 Failed download:
['ES2306=F']: YFTzMissingError('possibly delisted; no timezone found')


-> Download Complete.



/tmp/ipykernel_23949/377196534.py:19: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



KeyError: '^GSPC'